# <font color="blue">Text Analytics Tutorial</font>
**Aron Culotta**  
**Associate Professor of Computer Science**  
**Illinois Institute of Technology**  
*November 13, 2019*

# Classification

# Clustering